# Import Packages

In [1]:
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

import pandas_datareader.data as web
import polars as pl
import wrds
from fredapi import Fred
from datetime import date

from functions.utils.func import *

### Establish First Connection to WRDS

In [2]:
# db = wrds.Connection(wrds_username='jofan23')
# db.create_pgpass_file()

# Present Date

In [3]:
live = True
update_stock_list = True
current_date = date.today().strftime('%Y-%m-%d')

# Link Table

In [9]:
print("-" * 60)
sql_link = f"""
    SELECT a.gvkey, a.conm, a.tic, a.cusip, a.cik, a.sic, a.naics, b.linkprim,
           b.linktype, b.liid, b.lpermno, b.lpermco, b.linkdt, b.linkenddt
    FROM comp_na_daily_all.names as a
    INNER JOIN crsp.ccmxpf_lnkhist as b
    ON a.gvkey = b.gvkey
    WHERE b.linktype in ('LC', 'LU')
    AND b.linkprim in ('P', 'C')
    ORDER BY a.gvkey;
"""

# Read in linking table
print("Read in linking table...")
db = wrds.Connection(wrds_username='jofan23')
link_table = db.raw_sql(sql_link)
db.close()

# Rename columns
print("Rename columns...")
link_table = link_table.rename(columns={
    'linkdt': 'timeLinkStart_d',
    'linkenddt': 'timeLinkEnd_d',
    'lpermno': 'permno',
    'tic': 'ticker'
})

# Drop rows where the 'ticker' column has numbers
print("Drop rows where the 'ticker' column has numbers...")
link_table = link_table[~link_table['ticker'].str.contains('\d', regex=True)]

# Remove any non-alphabetic characters from the 'ticker' column
print("Remove any non-alphabetic characters from the 'ticker' column...")
link_table['ticker'] = link_table['ticker'].str.replace('[^A-Za-z]', '', regex=True)
link_table['permno'] = link_table['permno'].astype(int)

# Remove duplicate permno
print("Remove duplicate permno, gvkey, and ticker and keep the most recent based off timeLinkEnd_d...")
link_table['timeLinkEnd_d'] = pd.to_datetime(link_table['timeLinkEnd_d'].replace(pd.NaT, current_date))
link_table = link_table.sort_values(by=['ticker', 'timeLinkEnd_d'])
link_table = link_table.drop_duplicates(subset='ticker', keep='last')
link_table = link_table.drop_duplicates(subset='permno', keep='last')
link_table = link_table.drop_duplicates(subset='gvkey', keep='last')

# Export link table
print("Export link table...")
link_table.to_parquet(get_parquet_dir(live) / 'data_link.parquet.brotli', compression='brotli')

------------------------------------------------------------
Read in linking table...
Loading library list...
Done
Rename columns...
Drop rows where the 'ticker' column has numbers...
Remove any non-alphabetic characters from the 'ticker' column...
Remove duplicate permno, gvkey, and ticker and keep the most recent based off timeLinkEnd_d...
Export link table...


# CRSP Daily

### This will be used to determine what stocks to trade on an annual basis

In [135]:
if update_stock_list:
    # Read in CRSP dataset
    print('Read in CRSP dataset')
    crsp = pd.read_csv(get_large_dir(live) / 'crsp_price.csv')
    
    # Rename Columns
    print('Rename columns...')
    crsp.columns = crsp.columns.str.lower()
    crsp = crsp.rename(columns={'prc':'Close', 'bid':'High', 'ask':'Low', 'openprc':'Open', 'shrout':'outstanding', 'vol':'Volume', 'cfacpr':'adj_price'})
    
    # Adjust closing price
    print('Adjusting Close Price...')
    crsp['Close'] = crsp['Close']/crsp['adj_price']
    
    # Set and sort index
    print('Set and sort indices...')
    crsp.date = pd.to_datetime(crsp.date)
    crsp = crsp.set_index(['permno', 'date'])
    crsp = crsp.sort_index(level=['permno', 'date'])
    
    # Remove duplicate indices and replace all infinity with nan
    print('Remove duplicate indices and infinity...')
    crsp = crsp[~crsp.index.duplicated(keep='first')]
    crsp = crsp.replace([np.inf, -np.inf], np.nan)
    
    # Remove stocks that have more than 1 NAN values in their Closing price column
    # Stocks that get delisted have 1 row of NAN values as their last row
    # Stocks that switch ticker (WM to COOP: 81593) have rows of NAN valuescap = cap.dropna(subset='Close')
    # Afterwards, drop all rows that have NAN values in Close (every delisted permno stock only has 1 NAN in Close now)
    print('Remove stocks with NAN...')
    nan_counts = crsp.groupby('permno')['Close'].apply(lambda x: x.isna().sum())
    valid_permnos = nan_counts[nan_counts <= 1].index.tolist()
    crsp = crsp[crsp.index.get_level_values('permno').isin(valid_permnos)]
    crsp = crsp.dropna(subset='Close')
    
    # Remove dates in stocks that have a negative closing price
    crsp = crsp[crsp['Close'] >= 0]
    
    # Remove stocks that do not have at least 3 years worth of year data
    print('Set length to 3 years...')
    crsp = set_length(crsp, 3)
    
    # Drop permno that do not have over 5B market cap
    print("Drop permno that do not have over 5B market cap...")
    crsp['market_cap'] = crsp['Close'] * crsp['outstanding'] * 1000
    avg_cap = crsp.groupby('permno')['market_cap'].mean()
    above_cap = avg_cap[avg_cap > 5_000_000_000].index
    crsp = crsp[crsp.index.get_level_values('permno').isin(above_cap)]

    # Drop permno/date pairs that have multiple tickers
    print("Drop permno/date pairs that have multiple tickers...")
    ticker_counts = crsp.groupby('permno')['ticker'].nunique()
    to_drop_permnos = ticker_counts[ticker_counts > 1].index
    crsp = crsp[~crsp.index.get_level_values('permno').isin(to_drop_permnos)]
    
    # Export list of stocks
    print('Exporting stock list...')
    print(f'Number of stocks: {len(get_stock_idx(crsp))}')
    export_stock(crsp, get_large_dir(live) / 'permno_crsp.csv')

Drop permno/date pairs that have multiple tickers...
Exporting stock list...
Number of stocks: 1110


# Compustat Quarterly

In [11]:
print("-" * 60)
sql_compustat_quarterly = f"""
    SELECT a.gvkey, a.datadate, a.fyearq, a.fqtr, a.datacqtr, a.datafqtr, a.acoq,
	a.actq, a.ajexq, a.apq, a.atq, a.ceqq, a.cheq, a.cogsq, a.cshoq, a.cshprq,
	a.dlcq, a.dlttq, a.dpq, a.drcq, a.drltq, a.dvpsxq, a.dvpq, a.dvy, a.epspiq, a.epspxq, a.fopty,
	a.gdwlq, a.ibq, a.invtq, a.intanq, a.ivaoq, a.lcoq, a.lctq, a.loq, a.ltq, a.mibq,
	a.niq, a.oancfy, a.oiadpq, a.oibdpq, a.piq, a.ppentq, a.ppegtq, a.prstkcy, a.prccq,
	a.pstkq, a.rdq, a.req, a.rectq, a.revtq, a.saleq, a.seqq, a.sstky, a.txdiq,
	a.txditcq, a.txpq, a.txtq, a.xaccq, a.xintq, a.xsgaq, a.xrdq, a.capxy
    FROM comp_na_daily_all.fundq as a
	WHERE a.consol = 'C'
	AND a.popsrc = 'D'
	AND a.datafmt = 'STD'
	AND a.curcdq = 'USD'
	AND a.indfmt = 'INDL'
    AND a.datadate BETWEEN '2005-01-01' AND '{current_date}'
"""

# Read in Compustat Quarterly
print("Read In Compustat Quarterly...")
db = wrds.Connection(wrds_username='jofan23')
compustat_quarterly = db.raw_sql(sql_compustat_quarterly)
db.close()

# Read in link table
print("Read in link table...")
link_table = pd.read_parquet(get_parquet_dir(live) / 'data_link.parquet.brotli')
link_table = link_table.drop(['cusip', 'conm'], axis=1)

# Merge link table and Compustat Quarterly
print("Merge link table and Compustat Quarterly...")
quarterly = compustat_quarterly.merge(link_table, on='gvkey', how='inner')

# Keep only the most recent data for each fiscal quarter
print("Keep only the most recent data for each fiscal quarter...")
quarterly = quarterly.sort_values(by=['gvkey', 'fyearq', 'fqtr', 'datadate'])
quarterly = quarterly.groupby(['gvkey', 'fyearq', 'fqtr']).last().reset_index()

# Convert to datetime
print("Convert to datetime...")
quarterly['datadate'] = pd.to_datetime(quarterly['datadate'])
quarterly['rdq'] = pd.to_datetime(quarterly['rdq'])

# Shift data 3 months forward
print("Shift data 3 months forward...")
quarterly['time_avail_m'] = (quarterly['datadate'] + pd.DateOffset(months=3)).dt.to_period('M')
quarterly.loc[(~quarterly['rdq'].isnull()) & (quarterly['rdq'].dt.to_period('M') > quarterly['time_avail_m']), 'time_avail_m'] = quarterly['rdq'].dt.to_period('M')

# Compute month difference
print("Compute month difference...")
month_diff = (quarterly['rdq'] - quarterly['datadate']).dt.days // 30
quarterly = quarterly.drop(quarterly[(month_diff > 6) & ~quarterly['rdq'].isnull()].index)
quarterly = quarterly.sort_values(by=['gvkey', 'time_avail_m', 'datadate'])

# Keep most recent data
print("Keep most recent data...")
quarterly = quarterly.groupby(['gvkey', 'time_avail_m']).last().reset_index()

# Create extra yearly columns
print("Create extra yearly columns...")
for col in ['sstky', 'prstkcy', 'oancfy', 'fopty']:
    grouped = quarterly.groupby(['gvkey', 'fyearq'])[col]
    condition = quarterly['fqtr'] == 1
    new_values = np.where(condition, quarterly[col], quarterly[col] - grouped.shift(1))
    quarterly[col + 'q'] = new_values
    
# Convert index from quarterly to monthly
print("Convert index from quarterly to monthly...")
quarterly = quarterly.loc[quarterly.index.repeat(3)]
quarterly['tempTimeAvailM'] = quarterly['time_avail_m']
quarterly = quarterly.sort_values(by=['gvkey', 'tempTimeAvailM'])
quarterly['time_avail_m'] = quarterly.groupby(['gvkey', 'tempTimeAvailM']).cumcount() + quarterly['time_avail_m']

# Sort values
print("Sort values and keep most recent data...")
quarterly = quarterly.sort_values(by=['gvkey', 'time_avail_m', 'datadate'])
# Keep most recent data
quarterly = quarterly.groupby(['gvkey', 'time_avail_m']).last().reset_index()
quarterly = quarterly.drop(columns=['tempTimeAvailM'])
quarterly = quarterly.rename(columns={'datadate': 'datadateq', 'time_avail_m':'date'})

# Convert from YY-MM to YY-MM-DD (2012-01 to 2012-01-31)
print("Convert from YY-MM to YY-MM-DD (2012-01 to 2012-01-31)...")
quarterly.date = quarterly.date.dt.to_timestamp("M")
quarterly = quarterly.set_index(['permno', 'date'])
quarterly = quarterly.sort_index(level=['permno', 'date'])

# Read in CRSP stock list
if update_stock_list:
    print("Read in CRSP stock list...")
    stock = read_stock(get_large_dir(live) / 'permno_crsp.csv')
    quarterly = get_stocks_data(quarterly, stock)
    print("Exporting stock list for live trading...")
    print(f'Number of stocks: {len(get_stock_idx(quarterly))}')
    export_stock(quarterly, get_large_dir(live) / 'permno_compustat.csv')

    # Export ticker list
    print("Export ticker list...")
    ticker = link_table[link_table['permno'].isin(get_stock_idx(quarterly))]
    ticker = ticker.set_index('ticker')
    export_stock(ticker, get_large_dir(live) / 'ticker_compustat.csv')
    
# Convert data to numerical format (exclude columns that are not numerical format)
print("Convert data to numerical format (exclude columns that are not numerical format)...")
numeric_cols = quarterly.select_dtypes(include=['number']).columns
quarterly[numeric_cols] = quarterly[numeric_cols].astype(float)
non_numeric_cols = quarterly.select_dtypes(exclude=['number']).columns
quarterly_numeric = quarterly[numeric_cols]
quarterly_numeric = quarterly_numeric.sort_index(level=['permno', 'date'])

# Forward fill yearly data
print("Forward fill yearly data...")
cols_to_fill = [col for col in quarterly_numeric.columns if col.endswith('y')]
quarterly_numeric[cols_to_fill] = quarterly_numeric[cols_to_fill].ffill()

# Export data
print("Export data...")
quarterly_numeric.to_parquet(get_parquet_dir(live) / 'data_fund_raw_q.parquet.brotli', compression='brotli')

------------------------------------------------------------
Read In Compustat Quarterly...
Loading library list...
Done
Read in link table...
Merge link table and Compustat Quarterly...
Keep only the most recent data for each fiscal quarter...
Convert to datetime...
Shift data 3 months forward...
Compute month difference...
Keep most recent data...
Create extra yearly columns...
Convert index from quarterly to monthly...
Sort values and keep most recent data...
Convert from YY-MM to YY-MM-DD (2012-01 to 2012-01-31)...
Read in CRSP stock list...
Exporting stock list for live trading...
Number of stocks: 856
Export ticker list...
Convert data to numerical format (exclude columns that are not numerical format)...
Forward fill yearly data...
Export data...


# Compustat Annual

In [12]:
print("-" * 60)
sql_compustat_annual = f"""
    SELECT a.gvkey, a.datadate, a.conm, a.fyear, a.tic, a.cusip, a.naicsh, a.sich, 
    a.aco, a.act, a.ajex, a.am, a.ao, a.ap, a.at, a.capx, a.ceq, a.ceqt, a.che, a.cogs,
    a.csho, a.cshrc, a.dcpstk, a.dcvt, a.dlc, a.dlcch, a.dltis, a.dltr,
    a.dltt, a.dm, a.dp, a.drc, a.drlt, a.dv,a.dvc,a.dvp,a.dvpa,a.dvpd,
    a.dvpsx_c, a.dvt, a.ebit, a.ebitda, a.emp, a.epspi, a.epspx, a.fatb, a.fatl,
    a.ffo, a.fincf, a.fopt, a.gdwl, a.gdwlia, a.gdwlip, a.gwo, a.ib, a.ibcom,
    a.intan, a.invt, a.ivao, a.ivncf, a.ivst, a.lco, a.lct, a.lo ,a.lt, a.mib,
    a.msa, a.ni, a.nopi, a.oancf, a.ob, a.oiadp, a.oibdp, a.pi, a.ppenb, a.ppegt,
    a.ppenls, a.ppent, a.prcc_c, a.prcc_f, a.prstkc, a.prstkcc, a.pstk, a.pstkl, a.pstkrv,
    a.re, a.rect, a.recta, a.revt, a.sale, a.scstkc, a.seq, a.spi, a.sstk,
    a.tstkp, a.txdb, a.txdi, a.txditc, a.txfo, a.txfed, a.txp, a.txt,
    a.wcap, a.wcapch, a.xacc, a.xad, a.xint, a.xrd, a.xpp, a.xsga
    FROM comp_na_daily_all.funda as a
    WHERE a.consol = 'C'
    AND a.popsrc = 'D'
    AND a.datafmt = 'STD'
    AND a.curcd = 'USD'
    AND a.indfmt = 'INDL'
    AND a.datadate BETWEEN '2005-01-01' AND '{current_date}'
"""

# Read in Compustat Annual
print("Read in Compustat Annual...")
db = wrds.Connection(wrds_username='jofan23')
compustat_annual = db.raw_sql(sql_compustat_annual)
db.close()

# Read in link table
print("Read in link table...")
link_table = pd.read_parquet(get_parquet_dir(live) / 'data_link.parquet.brotli')
link_table = link_table.drop(['cusip', 'conm'], axis=1)

# Merge link table and Compustat Annual
print("Merge link table and Compustat Annual...")
annual = compustat_annual.merge(link_table, on='gvkey', how='inner')

# Drop rows based on condition
print("Drop rows based on condition...")
annual = annual.dropna(subset=['at', 'prcc_c', 'ni'])

# Extract 6 digits from CUSIP
print("Extract 6 digits from CUSIP...")
annual['cnum'] = annual['cusip'].str[:6]

# Replacing missing values
print("Replacing missing values...")
annual['dr'] = annual.apply(lambda row: row['drc'] + row['drlt'] if pd.notna(row['drc']) and pd.notna(row['drlt']) else (row['drc'] if pd.notna(row['drc']) else (row['drlt'] if pd.notna(row['drlt']) else None)), axis=1)
annual.loc[(annual['dcpstk'] > annual['pstk']) & pd.notna(annual['dcpstk']) & pd.notna(annual['pstk']) & pd.isna(annual['dcvt']), 'dc'] = annual['dcpstk'] - annual['pstk']
annual.loc[pd.isna(annual['pstk']) & pd.notna(annual['dcpstk']) & pd.isna(annual['dcvt']), 'dc'] = annual['dcpstk']
annual.loc[pd.isna(annual['dc']), 'dc'] = annual['dcvt']
annual['xint0'] = annual['xint'].fillna(0)
annual['xsga0'] = annual['xsga'].fillna(0)
annual['xad0'] = annual.apply(lambda row: 0 if row['xad'] < 0 else row['xad'], axis=1)
vars_list = ['nopi', 'dvt', 'ob', 'dm', 'dc', 'aco', 'ap', 'intan', 'ao', 'lco', 'lo', 'rect', 'invt', 'drc', 'spi', 'gdwl', 'che', 'dp', 'act', 'lct', 'tstkp', 'dvpa', 'scstkc', 'sstk', 'mib', 'ivao', 'prstkc', 'prstkcc', 'txditc', 'ivst']
for var in vars_list:
    annual[var].fillna(0, inplace=True)

# Shift data forward by 6 months
print("Shift data forward by 6 months...")
annual['date'] = pd.to_datetime(annual['datadate']).dt.to_period('M') + 6

# Convert index from annually to monthly
print("Convert index from annually to monthly...")
annual = annual.reindex(annual.index.repeat(12))
annual['tempTime'] = annual.groupby(['gvkey', 'date']).cumcount()
annual['date'] += annual['tempTime']
annual = annual.drop(columns=['tempTime'])

# Convert from YY-MM to YY-MM-DD (2012-01 to 2012-01-31)
print("Convert from YY-MM to YY-MM-DD (2012-01 to 2012-01-31)")
annual.date = annual.date.dt.to_timestamp("M")
annual = annual.drop('datadate', axis=1)

# Set index and remove duplicate indices
print("Set index and remove duplicate indices...")
annual = annual.set_index(['permno', 'date'])
annual = annual.sort_index(level=['permno', 'date'])
annual = annual[~annual.index.duplicated(keep='first')]

# Retrieve live trade stock list
print("Retrieve live trade stock list...")
stock = read_stock(get_large_dir(live) / 'permno_compustat.csv')
annual = get_stocks_data(annual, stock)

# Export data
print("Export data")
annual.to_parquet(get_parquet_dir(live) / 'data_fund_raw_a.parquet.brotli', compression='brotli')

------------------------------------------------------------
Read in Compustat Annual...
Loading library list...
Done
Read in link table...
Merge link table and Compustat Annual...
Drop rows based on condition...
Extract 6 digits from CUSIP...
Replacing missing values...
Shift data forward by 6 months...
Convert index from annually to monthly...
Convert from YY-MM to YY-MM-DD (2012-01 to 2012-01-31)
Set index and remove duplicate indices...
Retrieve live trade stock list...
Export data


# Live Market Data

In [13]:
print("-" * 60)
ticker_list = read_stock(get_large_dir(live) / 'ticker_compustat.csv')

# Read in live market data
print("Read in live market data...")
price = yf.download(ticker_list, start='2005-01-01', end=current_date)
price = price.stack().swaplevel().sort_index()
price.index.names = ['ticker', 'date']
price = price.astype(float)

------------------------------------------------------------
Read in live market data...
[*********************100%%**********************]  856 of 856 completed


111 Failed downloads:
['FBC', 'CMMCY', 'ZSANQ', 'WINMQ', 'FDC', 'BASXQ', 'CXO', 'ZEN', 'IVANF', 'LM', 'LN', 'SDLPQ', 'CELG', 'SC', 'SCON', 'WHCI', 'QTNTQ', 'AVP', 'TSS', 'NRTLQ', 'BFB', 'FTSI', 'PIRRQ', 'CLR', 'Y', 'BPY', 'KL', 'STOR', 'CPPRQ', 'CDK', 'VAR', 'SQBGQ', 'PSXP', 'CTXS', 'WPSL', 'BRKB', 'FRANQ', 'TIF', 'COUP', 'CERN', 'TWTR', 'NBL', 'XEC', 'ATH', 'TTPH', 'RTN', 'AIBYY', 'RRD', 'XLNX', 'WPGGQ', 'APC', 'GG', 'MDRIQ', 'NLSN', 'NIHD', 'WLL', 'DRE', 'IOGPQ', 'IAA', 'PBCT', 'CHNG', 'BPL', 'AVLR', 'JENGQ', 'KSU', 'ALXN', 'ZAYO', 'AUY', 'QTWWQ', 'STI', 'PRAH']: Exception('%ticker%: No timezone found, symbol may be delisted')
['DPS', 'LEHMQ', 'FDO', 'WWAV', 'THQIQ', 'SCMR', 'LLTC', 'GNVC', 'ELN', 'HCBK', 'HSP', 'CAM', 'TWC', 'CFN', 'XTO', 'STJ', 'GENZ', 'LVLT', 'ARG', 'THMRQ', 'KMP', 'RAI', 'WYE', 'SPLS', 'SWY', 'LNKD', 'BRCM', 'TLM', 'EPB', 'APOL', 'BCR', 'SIAL', 'SNDK', 'LO', 'AGU', 'MJN', 'FMD', 'FSL', 'EDS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 

In [14]:
# Read in link table
print("Read in link table...")
link_table = pd.read_parquet(get_parquet_dir(live) / 'data_link.parquet.brotli')

# Merge link table and Live Market Price
print("Merge link table and Live Market Price...")
price = price.reset_index()
price = price.merge(link_table, on='ticker', how='inner')
price = price.reset_index().set_index(['permno', 'date'])
price = price.sort_index(level=['permno', 'date'])

# Remove duplicate indices (there should be none)
print("Remove duplicate indices (there should be none)...")
price = price[~price.index.duplicated(keep='first')]

# Export ohclv
print('Exporting Price...')
ohclv = price[['Open', 'High', 'Close', 'Low', 'Volume']]
ohclv.to_parquet(get_parquet_dir(live) / 'data_price.parquet.brotli', compression='brotli')

# Export date
print('Export Date...')
date = price.drop(columns=price.columns)
date.to_parquet(get_parquet_dir(live) / 'data_date.parquet.brotli', compression='brotli')

# Export ticker
print('Exporting Tickers...')
ticker = price[['ticker']]
ticker.to_parquet(get_parquet_dir(live) / 'data_ticker.parquet.brotli', compression='brotli')

print("Exporting permno list for live trading...")
print(f'Number of stocks: {len(get_stock_idx(ohclv))}')
export_stock(ohclv, get_large_dir(live) / 'permno_live.csv')

print("Exporting ticker list for live trading...")
ticker = ticker.reset_index()
ticker = ticker.set_index('ticker')
print(f'Number of stocks: {len(get_stock_idx(ticker))}')
export_stock(ticker, get_large_dir(live) / 'ticker_live.csv')

Read in link table...
Merge link table and Live Market Price...
Remove duplicate indices (there should be none)...
Exporting Price...
Export Date...
Exporting Tickers...
Exporting permno list for live trading...
Number of stocks: 745
Exporting ticker list for live trading...
Number of stocks: 745


# Compustat Pension

In [16]:
print("-" * 60)
sql_compustat_pension = f"""
    SELECT a.gvkey, a.datadate, a.paddml, a.pbnaa, a.pbnvv, a.pbpro, 
	       a.pbpru, a.pcupsu, a.pplao, a.pplau
    FROM comp_na_daily_all.aco_pnfnda as a
	WHERE a.consol = 'C'
	AND a.popsrc = 'D'
	AND a.datafmt = 'STD'
	AND a.indfmt = 'INDL'
    AND a.datadate BETWEEN '2005-01-01' AND '{current_date}'
"""

# Read in Pension Annual
print("Read in Pension Annual...")
db = wrds.Connection(wrds_username='jofan23')
pension = db.raw_sql(sql_compustat_pension)
db.close()

# Drop duplicate indices
print("Drop duplicate indices...")
pension = pension.sort_values(by=['gvkey', 'datadate'])
pension = pension.groupby(['gvkey', 'datadate']).last().reset_index()

# Convert to datetime and set index
print("Convert to datetime and set index...")
pension['datadate'] = pd.to_datetime(pension['datadate'])
pension = pension.rename(columns = {'datadate': 'date', 'tic': 'ticker'})
pension = pension.set_index('date')

# Shift everything 1 year forward
print("Shift everything 1 year forward...")
for col in pension.columns:
    if col != 'gvkey' or col != 'indfmt' or col != 'datafmt' or col != 'consol' or col != 'popsrc' or col != 'ticker':
        pension[col] = pension.groupby('gvkey')[col].shift(1)

# Export data
print("Export data...")
pension.to_parquet(get_parquet_dir(live) / 'data_pension.parquet.brotli', compression='brotli')

------------------------------------------------------------
Read in Pension Annual...
Loading library list...
Done
Drop duplicate indices...
Convert to datetime and set index...
Shift everything 1 year forward...
Export data...


# Compustat Industry

In [46]:
print("-" * 60)
sql_compustat_industry = f"""
    SELECT a.gvkey, a.gind, a.gsubind, a.sic
    FROM comp_na_daily_all.names as a
"""

sql_crsp_industry = f"""
    SELECT a.permno, a.siccd
    FROM crsp_a_stock.dsenames as a
"""

# Read in Compustat Industry
print("Read in Compustat Industry...")
db = wrds.Connection(wrds_username='jofan23')
industry_compustat = db.raw_sql(sql_compustat_industry)
industry_compustat = industry_compustat.rename(columns={'sic': 'sic_comp'})
industry_compustat['sic_comp'] = industry_compustat['sic_comp'].astype(int)
db.close()

# Read in CRSP Industry
print("Read in CRSP Industry...")
db = wrds.Connection(wrds_username='jofan23')
industry_crsp = db.raw_sql(sql_crsp_industry)
industry_crsp = industry_crsp.rename(columns={'siccd': 'sic_crsp'})
industry_crsp['sic_crsp'] = industry_crsp['sic_crsp'].astype(int)
db.close()

# Read in link table
print("Read in link table...")
link_table = pd.read_parquet(get_parquet_dir(live) / 'data_link.parquet.brotli')
link_table = link_table.drop(['conm', 'sic'], axis=1)

# Merge link table and Compustat Annual
print("Merge link table and Compustat Annual...")
industry = pd.merge(industry_compustat, link_table, on='gvkey', how='inner')
industry = industry.merge(industry_crsp, on='permno', how='inner')

# Rename Columns
print('Rename columns...')
industry.columns = industry.columns.str.lower()
industry = industry.rename(columns={'gsubind':'Subindustry', 'gind':'Industry'})

# Remove duplicate permno
print('Remove duplicate permno...')
industry = industry.drop_duplicates(subset='permno')
industry = industry[['permno', 'Industry', 'Subindustry', 'sic_crsp', 'sic_comp']]

# Assign Fama industries based off given ranges
print("Assign Fama industries based off given ranges...")
def assign_ind(df, column_name, sic_ranges, label):
    # Sic from CRSP and Compustat
    df['sic_temp_crsp'] = df['sic_crsp']
    df['sic_temp_comp'] = df['sic_comp']

    # Iterate through each range and assign industry
    for r in sic_ranges:
        if isinstance(r, tuple):
            df.loc[(df['sic_temp_crsp'] >= r[0]) & (df['sic_temp_crsp'] <= r[1]), f'{column_name}_crsp'] = label
            df.loc[(df['sic_temp_comp'] >= r[0]) & (df['sic_temp_comp'] <= r[1]), f'{column_name}_comp'] = label
        else:
            df.loc[df['sic_temp_crsp'] == r, f'{column_name}_crsp'] = label
            df.loc[df['sic_temp_comp'] == r, f'{column_name}_comp'] = label
    
    df = df.drop(columns=['sic_temp_crsp', 'sic_temp_comp'])
    return df
    
# FF49 Industry ranges
fama_ind = {
'agric': [(100, 199), (200, 299), (700, 799), (910, 919), 2048],
'food': [(2000, 2009), (2010, 2019), (2020, 2029), (2030, 2039), (2040, 2046), (2050, 2059), (2060, 2063), (2070, 2079), (2090, 2092), 2095, (2098, 2099)],
'soda': [(2064, 2068), 2086, 2087, 2096, 2097],
'beer': [2080, 2082, 2083, 2084, 2085],
'smoke': [(2100, 2199)],
'toys': [(920, 999), (3650, 3651), 3652, 3732, (3930, 3931), (3940, 3949)],
'fun': [(7800, 7829), (7830, 7833), (7840, 7841), 7900, (7910, 7911), (7920, 7929), (7930, 7933), (7940, 7949), 7980, (7990, 7999)],
'books': [(2700, 2709), (2710, 2719), (2720, 2729), (2730, 2739), (2740, 2749), (2770, 2771), (2780, 2789), (2790, 2799)],
'hshld': [2047, (2391, 2392), (2510, 2519), (2590, 2599), (2840, 2843), 2844, (3160, 3161), (3170, 3171), 3172, (3190, 3199), 3229, 3260, (3262, 3263), 3269, (3230, 3231), (3630, 3639), (3750, 3751), 3800, (3860, 3861), (3870, 3873), (3910, 3911), 3914, 3915, (3960, 3962), 3991, 3995],
'clths': [(2300, 2390), (3020, 3021), (3100, 3111), (3130, 3131), (3140, 3149), (3150, 3151), (3963, 3965)],
'hlth': [(8000, 8099)],
'medeq': [3693, (3840, 3849), (3850, 3851)],
'drugs': [2830, 2831, 2833, 2834, 2835, 2836],
'chems': [(2800, 2809), (2810, 2819), (2820, 2829), (2850, 2859), (2860, 2869), (2870, 2879), (2890, 2899)],
'rubbr': [3031, 3041, (3050, 3053), (3060, 3069), (3070, 3079), (3080, 3089), (3090, 3099)],
'txtls': [(2200, 2269), (2270, 2279), (2280, 2284), (2290, 2295), 2297, 2298, 2299, (2393, 2395), (2397, 2399)],
'bldmt': [(800, 899), (2400, 2439), (2450, 2459), (2490, 2499), (2660, 2661), (2950, 2952), 3200, (3210, 3211), (3240, 3241), (3250, 3259), 3261, 3264, (3270, 3275), (3280, 3281), (3290, 3293), (3295, 3299), (3420, 3429), (3430, 3433), (3440, 3441), 3442, 3446, 3448, 3449, (3450, 3451), 3452, (3490, 3499), 3996],
'cnstr': [(1500, 1511), (1520, 1529), (1530, 1539), (1540, 1549), (1600, 1699), (1700, 1799)],
'steel': [3300, (3310, 3317), (3320, 3325), (3330, 3339), (3340, 3341), (3350, 3357), (3360, 3369), (3370, 3379), (3390, 3399)],
'fabpr': [3400, 3443, 3444, (3460, 3469), (3470, 3479)],
'mach': [(3510, 3519), (3520, 3529), 3530, 3531, 3532, 3533, 3534, 3535, 3536, 3538, (3540, 3549), (3550, 3559), (3560, 3569), 3580, 3581, 3582, 3585, 3586, 3589, (3590, 3599)],
'elceq': [3600, (3610, 3613), (3620, 3621), (3623, 3629), (3640, 3644), 3645, 3646, (3648, 3649), 3660, 3690, (3691, 3692), 3699],
'autos': [2296, 2396, (3010, 3011), 3537, 3647, 3694, 3700, 3710, 3711, 3713, 3714, 3715, 3716, 3792, (3790, 3791), 3799],
'aero': [3720, 3721, (3723, 3724), 3725, (3728, 3729)],
'ships': [(3730, 3731), (3740, 3743)],
'guns': [(3760, 3769), 3795, (3480, 3489)],
'gold': [(1040, 1049)],
'mines': [(1000, 1009), (1010, 1019), (1020, 1029), (1030, 1039), (1050, 1059), (1060, 1069), (1070, 1079), (1080, 1089), (1090, 1099), (1100, 1119), (1400, 1499)],
'coal': [(1200, 1299)],
'oil': [1300, (1310, 1319), (1320, 1329), (1330, 1339), (1370, 1379), 1380, 1381, 1382, 1389, (2900, 2912), (2990, 2999)],
'util': [4900, (4910, 4911), (4920, 4922), 4923, (4924, 4925), (4930, 4931), 4932, 4939, (4940, 4942)],
'telcm': [4800, (4810, 4813), (4820, 4822), (4830, 4839), (4840, 4841), 4880, 4890, 4891, 4892, 4899],
'persv': [(7020, 7021), (7030, 7033), 7200, (7210, 7212), 7214, (7215, 7216), 7217, 7219, (7220, 7221), (7230, 7231), (7240, 7241), (7250, 7251), (7260, 7269), (7270, 7290), 7291, (7292, 7299), 7395, 7500, (7520, 7529), (7530, 7539), (7540, 7549), 7600, 7620, 7622, 7623, 7629, 7630, 7640, (7690, 7699), (8100, 8199), (8200, 8299), (8300, 8399), (8400, 8499), (8600, 8699), (8800, 8899), (7510, 7515)],
'bussv': [(2750, 2759), 3993, 7218, 7300, (7310, 7319), (7320, 7329), (7330, 7339), (7340, 7342), 7349, (7350, 7351), 7352, 7353, 7359, (7360, 7369), 7374, 7376, 7377, 7378, 7379, 7380, (7381, 7382), 7383, 7384, 7385, 7389, 7390, 7391, (7392, 7392), 7393, 7394, 7396, 7397, 7399, (7519, 7519), 8700, (8710, 8713), (8720, 8721), (8730, 8734), (8740, 8748), (8900, 8910), 8911, (8920, 8999), (4220, 4229)],
'hardw': [(3570, 3579), 3680, 3681, 3682, 3683, 3684, 3685, 3686, 3687, 3688, 3689, 3695],
'softw': [(7370, 7372), 7375, 7373],
'chips': [3622, 3661, (3662, 3662), 3663, 3664, 3665, 3666, 3669, (3670, 3679), (3810, 3810), (3812, 3812)],
'labeq': [3811, 3820, 3821, 3822, 3823, 3824, 3825, 3826, 3827, 3829, (3830, 3839)],
'paper': [(2520, 2549), (2600, 2639), (2670, 2699), (2760, 2761), (3950, 3955)],
'boxes': [(2440, 2449), (2640, 2659), (3220, 3221), (3410, 3412)],
'trans': [(4000, 4013), (4040, 4049), 4100, (4110, 4119), (4120, 4121), (4130, 4131), (4140, 4142), (4150, 4151), (4170, 4173), (4190, 4199), 4200, (4210, 4219), (4230, 4231), (4240, 4249), (4400, 4499), (4500, 4599), (4600, 4699), 4700, (4710, 4712), (4720, 4729), (4730, 4739), (4740, 4749), 4780, 4782, 4783, 4784, 4785, 4789],
'whlsl': [5000, (5010, 5015), (5020, 5023), (5030, 5039), (5040, 5042), 5043, 5044, 5045, 5046, 5047, 5048, 5049, (5050, 5059), 5060, 5063, 5064, 5065, (5070, 5078), 5080, 5081, 5082, 5083, 5084, 5085, (5086, 5087), 5088, 5090, (5091, 5092), 5093, 5094, 5099, 5100, (5110, 5113), (5120, 5122), (5130, 5139), (5140, 5149), (5150, 5159), (5160, 5169), (5170, 5172), (5180, 5182), (5190, 5199)],
'rtail': [5200, (5210, 5219), (5220, 5229), (5230, 5231), (5250, 5251), (5260, 5261), (5270, 5271), 5300, 5310, 5320, (5330, 5331), 5334, (5340, 5349), (5390, 5399), 5400, (5410, 5411), 5412, (5420, 5429), (5430, 5439), (5440, 5449), (5450, 5459), (5460, 5469), (5490, 5499), 5500, (5510, 5529), (5530, 5539), (5540, 5549), (5550, 5559), (5560, 5569), (5570, 5579), (5590, 5599), (5600, 5699), 5700, (5710, 5719), (5720, 5722), (5730, 5733), 5734, 5735, 5736, (5750, 5799), 5900, (5910, 5912), (5920, 5929), (5930, 5932), 5940, 5941, 5942, 5943, 5944, 5945, 5946, 5947, 5948, 5949, (5950, 5959), (5960, 5969), (5970, 5979), (5980, 5989), 5990, 5992, 5993, 5994, 5995, 5999],
'meals': [(5800, 5819), (5820, 5829), (5890, 5899), 7000, (7010, 7019), (7040, 7049), 7213],
'banks': [6000, (6010, 6019), 6020, 6021, 6022, 6023, 6025, 6026, 6027, (6028, 6029), (6030, 6036), (6040, 6059), (6060, 6062), (6080, 6082), (6090, 6099), 6100, (6110, 6111), (6112, 6113), (6120, 6129), (6130, 6139), (6140, 6149), (6150, 6159), (6160, 6169), (6170, 6179), (6190, 6199)],
'insur': [6300, (6310, 6319), (6320, 6329), (6330, 6331), (6350, 6351), (6360, 6361), (6370, 6379), (6390, 6399), (6400, 6411)],
'rlest': [6500, 6510, 6512, 6513, 6514, 6515, (6517, 6519), (6520, 6529), (6530, 6531), 6532, (6540, 6541), (6550, 6553), (6590, 6599), (6610, 6611)],
'fin': [(6200, 6299), 6700, (6710, 6719), (6720, 6722), 6723, 6724, 6725, 6726, (6730, 6733), (6740, 6779), 6790, 6791, 6792, 6793, 6794, 6795, 6798, 6799],
'other': [(4950, 4959), (4960, 4961), (4970, 4971), (4990, 4991)]
}

# Iterate through each key
print("Iterate through each key...")
for name, ranges in fama_ind.items():
    print('-'*60)
    print(name)
    combined = assign_ind(industry, 'IndustryFama', ranges, name)

# Assign industry based off Compustat. If Compustat is NAN, then use CRSP
print("Assign industry based off Compustat. If Compustat is NAN, then use CRSP...")
industry['IndustryFama'] = industry['IndustryFama_comp'].combine_first(industry['IndustryFama_crsp'])
industry['IndustryFama'], category_mapping = industry['IndustryFama'].factorize()

# Fill NAN values for industries with -1
print("Fill industries with -1...")
cols_to_fill = ['Industry', 'Subindustry', 'IndustryFama']
industry = industry[cols_to_fill].fillna(-1)

# Retrieve live trade stock list
print("Retrieve live trade stock list...")
stock = read_stock(get_large_dir(live) / 'permno_live.csv')
industry = industry[industry['permno'].isin(stock)]

# Merge ind data with price dataset
print("Merge ind data with price dataset...")
date = pd.read_parquet(get_parquet_dir(live) / 'data_date.parquet.brotli')
date = date.reset_index(level='permno')
industry = pd.merge(date, industry, on=['permno'], how='inner')
industry = industry.reset_index().set_index(['permno', 'date']).sort_index(level=['permno', 'date'])
industry = industry.groupby('permno').ffill()
industry = industry[~industry.index.duplicated(keep='first')]
industry = industry.replace([np.inf, -np.inf], np.nan)

# Export industry
print("Exporting Industries...")
industry[['Industry']].to_parquet(get_parquet_dir(live) / 'data_ind.parquet.brotli', compression='brotli')
industry[['Subindustry']].to_parquet(get_parquet_dir(live) / 'data_ind_sub.parquet.brotli', compression='brotli')
industry[['IndustryFama']].to_parquet(get_parquet_dir(live) / 'data_ind_fama.parquet.brotli', compression='brotli')

------------------------------------------------------------
Read in Compustat Industry...
Loading library list...
Done
Read in CRSP Industry...
Loading library list...
Done
Read in link table...
Merge link table and Compustat Annual...
Rename columns...
Remove duplicate permno...
Assign Fama industries based off given ranges
Iterate through each key...
------------------------------------------------------------
agric
------------------------------------------------------------
food
------------------------------------------------------------
soda
------------------------------------------------------------
beer
------------------------------------------------------------
smoke
------------------------------------------------------------
toys
------------------------------------------------------------
fun
------------------------------------------------------------
books
------------------------------------------------------------
hshld
-----------------------------------------------

# Fama

In [49]:
# Get Fama data from web
fama_data = (web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', start=2005)[0].rename(columns={'Mkt-RF': 'MARKET'}))
fama_data.index.names = ['date']
fama_data = fama_data.astype(float)
fama_data.to_parquet(get_parquet_dir(live) / 'data_fama.parquet.brotli', compression='brotli')

# Macro

In [70]:
# API key
fred = Fred(api_key='00e07a5c98e913ea393c3b1f089e21d1')

# Read in Median CPI
print("Read in Median CPI...")
medianCPI = fred.get_series("MEDCPIM158SFRBCLE").to_frame()
medianCPI = medianCPI.reset_index()
medianCPI.columns = ['date', 'medCPI']

# Export data
print("Export data...")
medianCPI.to_csv(get_large_dir(True) / 'macro' / 'medianCPI.csv')

Read in Median CPI...
